# 3-3. Segmentation

In [ ]:
!pip3 install --upgrade mxnet-cu101 > /dev/null
!pip3 install --upgrade gluoncv > /dev/null

In [ ]:
!pip3 install --upgrade youtube-dl > /dev/null

In [ ]:
def get_youtube_video(youtube_url):
  import youtube_dl

  ydl_opts = {  
    # 'format': 'mp4' 
    'format': 'best[height<=480][ext=mp4]' 
    } 
  with youtube_dl.YoutubeDL(ydl_opts) as ydl: 
      info_dict = ydl.extract_info(youtube_url, download=True)    
      filename = ydl.prepare_filename(info_dict)
  return filename

## video download

In [ ]:
# country_road = get_youtube_video('https://www.youtube.com/watch?v=CObyXl__sEs')
# print(country_road)

In [ ]:
city_road = get_youtube_video('https://www.youtube.com/watch?v=xNGo9shCeh8')
print(city_road)

In [ ]:
# olympic = get_youtube_video('https://www.youtube.com/watch?v=agRTzCQXDQo')
# print(olympic)

In [ ]:
incheon = get_youtube_video('https://www.youtube.com/watch?v=rSOgnrBsStU')
print(incheon)

In [ ]:
hanam = get_youtube_video('https://www.youtube.com/watch?v=CDF5NjRhHOA')
print(hanam)

## read

In [ ]:
def read_video(filename, output_path, start_ms = 0, end_ms = None, step=1):
  from tqdm.notebook import tqdm
  import os 
  from gluoncv.utils.filesystem import try_import_cv2
  cv2 = try_import_cv2()
  # video_frames = [] 

  if not os.path.exists(output_path):
        os.makedirs(output_path)

  cap = cv2.VideoCapture(filename)

  cap.set(cv2.CAP_PROP_POS_MSEC, start_ms);

  width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  fps = cap.get(cv2.CAP_PROP_FPS)
  total_ms = frame_count * fps * 1000
  print('Frame width:', width)
  print('Frame height:', height)
  print('Frame count:', frame_count)
  print('FPS:', fps)
  print('total sec:', int(total_ms/1000))

  capture_count = 0
  
  if end_ms != None:
    time_gap_sec = (end_ms - start_ms)/1000
    pbar_count = round(time_gap_sec*fps)
  else:
    pbar_count = round(frame_count)

  with tqdm(total=pbar_count) as pbar:
    while cap.isOpened():
        ret, img = cap.read()
        if not ret:
            cap.release()
            break 
        frame_no = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        frame_msec = int(cap.get(cv2.CAP_PROP_POS_MSEC))

        # print('{}. msec:{}, endmsec:{}'.format(frame_no, frame_msec, end_ms)) 
        if end_ms != None and frame_msec >= end_ms:
            break

        # print('{}. msec:{}'.format(frame_no, frame_msec)) 

        # video_frames.append(img)
        cv2.imwrite(os.path.join(output_path, '%05d.jpg'%(frame_no)), img)
        capture_count += 1
        pbar.update(step)
        if step > 1:
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_no + (step-1));
  
  print('capture_count:', capture_count)

  return width, height, fps, frame_count

In [ ]:
# !rm -rf city*

In [ ]:
width, height, fps, frame_count = read_video(city_road, './city_road', 1000*60*28.5, 1000*60*29, 1)

## 분석

In [ ]:
import mxnet as mx
from mxnet import image
from mxnet.gluon.data.vision import transforms
import gluoncv 

In [ ]:
psp = gluoncv.model_zoo.get_model('psp_resnet101_ade', pretrained=True)
# psp = gluoncv.model_zoo.get_model('psp_resnet101_citys', pretrained=True)

In [ ]:
def display_semantic_segmentation_mask(model, im_fname, height, color_pallete='ade20k'):
  import mxnet as mx
  from mxnet import image
  from mxnet.gluon.data.vision import transforms
  from matplotlib import pyplot as plt
  import gluoncv
  from gluoncv.utils.viz import get_color_pallete
  import matplotlib.image as mpimg
  from gluoncv.utils.filesystem import try_import_cv2
  cv2 = try_import_cv2()

  # using cpu
  ctx = mx.cpu(0) 
 
  # image loading(원본이미지를 그대로 불러옴.)
  from gluoncv import data 
  _, org_image = data.transforms.presets.yolo.load_test(im_fname, short=height)
  img = mx.nd.array(org_image) 

  # image pre transforming
  from gluoncv.data.transforms.presets.segmentation import test_transform
  img = test_transform(img, ctx)

  # inference
  output = model.predict(img)
  predict = mx.nd.squeeze(mx.nd.argmax(output, 1)).asnumpy()

  # display
  mask = get_color_pallete(predict, color_pallete)
  mask.save('output.png')
  mask_img = image.imread('output.png')
  
  #mask image와 원본이미지 합치기 투명도 0.8
  mask_img = gluoncv.utils.viz.cv_merge_two_images(org_image, mask_img, alpha=0.5)

  # cv BGR -> RGB
  mask_img = cv2.cvtColor(mask_img, cv2.COLOR_BGR2RGB)

  return mask_img

In [ ]:
im_fname = gluoncv.utils.download('https://miro.medium.com/max/700/1*hO_md3xNtcncjAmGrTwxYA.jpeg')
mask_img = display_semantic_segmentation_mask(psp, im_fname , 480)

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(15,15)) 
ax = gluoncv.utils.viz.plot_image(mask_img ) 
plt.show()

In [ ]:
def get_semantic_video(detector,image_path, output_path, output_filename, width, height, fps, step=1): 
  from tqdm.notebook import tqdm
  import os 
  from gluoncv.utils.filesystem import try_import_cv2
  cv2 = try_import_cv2()

  if not os.path.exists(output_path):
    os.makedirs(output_path) 

  file_list = os.listdir(image_path)
  file_list.sort() 
  
  fourcc = cv2.VideoWriter_fourcc(*'FMP4')

  out = cv2.VideoWriter('{}/{}'.format(output_path, output_filename), fourcc, fps/step, (width, height))
 
  index = 0 # image id(for skip)
  for filename in tqdm(file_list):
    # step 값으로 이미지 skip
    if (index % step) != 0: 
      index += 1
      continue
    try:
      im_file = '{}/{}'.format(image_path,filename) 
      img = display_semantic_segmentation_mask(detector, im_file,   height) 
      frame = cv2.resize(img, dsize=(width, height), interpolation=cv2.INTER_AREA)
      out.write(frame) 
    except Exception as ex:
      pass 
    index += 1 # image id(for skip)
  out.release()

In [ ]:
# !rm -rf city*

In [ ]:
get_semantic_video(psp, './city_road', 'city_detect', 'city.mp4', 320, 240, 30.0, 5)

In [ ]:
# !rm -rf hanam*

In [ ]:
width, height, fps, frame_count = read_video(hanam, './hanam_road', 1000*60*9.5, 1000*60*10, 1)

In [ ]:
get_semantic_video(psp, './hanam_road', 'hanam_detect', 'hanam.mp4', 320, 240, 29.970000648240262, 5)

In [ ]:
# !rm -rf incheon*
# !rm -rf city*
# !rm -rf hanam*

In [ ]:
width, height, fps, frame_count = read_video(incheon, './incheon_road', 1000*60*6.5, 1000*60*7, 4)

In [ ]:
get_semantic_video(psp, './incheon_road', 'incheon_detect', 'incheon.mp4', 320, 240, 30.0, 5)

# 실습

### 1. 아산 시내주행 드라이브 영상의 일부구간을 segmentation 영상으로 전환
- image 생성
- 이미지 처리하면서 detect 

In [ ]:
asan = get_youtube_video('https://www.youtube.com/watch?v=t1YEq_2Un1Y')
print(asan)

### 2. 미스터션샤인 하이라이트 영상의 일부구간을 segmentation 영상으로 전환
- image 생성
- 이미지 처리하면서 detect 


In [ ]:
sunshine = get_youtube_video('https://www.youtube.com/watch?v=4YTXB5qkGzI')
print(sunshine)